In [ ]:
import glob
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cmcrameri import cm  # noqa: F401
from weis.visualization.utils import load_OMsql_multi

plt.style.use("torque.mplstyle")

%matplotlib widget

## Load SQL Metadata

In [ ]:
# Load SQL metadata to link ranks/iterations with design variables
log_path = "../../../data/torque/laminar/log_laminar.sql*"
sql_data = load_OMsql_multi(log_path)

# Extract metadata into DataFrame
df_meta = pd.DataFrame(
    {
        "rank": sql_data["rank"],
        "iter": sql_data["iter"],
    }
)

# Extract any design variables that might be present
for key in sql_data.keys():
    if "tune_rosco_ivc" in key or "aeroelastic" in key:
        short_key = key.split(".")[-1]  # Get last part after dot
        df_meta[short_key] = sql_data[key]

print(f"Loaded metadata for {len(df_meta)} cases")
print(f"Available columns: {df_meta.columns.tolist()}")
print("\nFirst few rows:")
print(df_meta.head())

## Load Timeseries Pickle Files

In [ ]:
import pickle

# Base path for OpenFAST runs
base_path = "../../../data/torque/laminar/openfast_runs"

# Find all pickle files in timeseries subdirectories
pickle_pattern = os.path.join(base_path, "rank_*/iteration_*/timeseries/*.p")
pickle_files = sorted(glob.glob(pickle_pattern))

print(f"Found {len(pickle_files)} pickle files")

# Load all timeseries data
timeseries_data = {}

for pkl_file in pickle_files:
    # Extract rank and iteration from path
    match = re.search(r"rank_(\d+)/iteration_(\d+)", pkl_file)
    if not match:
        continue

    rank = int(match.group(1))
    iteration = int(match.group(2))

    # Load pickle file
    with open(pkl_file, "rb") as f:
        df_ts = pickle.load(f)

    # Store by (rank, iter) tuple
    timeseries_data[(rank, iteration)] = df_ts

    if len(timeseries_data) == 1:
        print(f"\nFirst file loaded from: {pkl_file}")
        print(f"DataFrame shape: {df_ts.shape}")
        print(f"Available columns: {df_ts.columns.tolist()[:20]}...")  # Show first 20

print(f"\nLoaded {len(timeseries_data)} timeseries DataFrames")
print(f"Keys (rank, iter): {sorted(timeseries_data.keys())}")

## Create Easy Access Structure

In [ ]:
# Merge SQL metadata with timeseries data for easy access
cases = []

for (rank, iteration), df_ts in timeseries_data.items():
    # Get metadata for this case
    meta_row = df_meta[(df_meta["rank"] == rank) & (df_meta["iter"] == iteration)]

    if len(meta_row) == 0:
        print(f"Warning: No metadata found for rank={rank}, iter={iteration}")
        continue

    # Create case dict with metadata and timeseries
    case = {
        "rank": rank,
        "iter": iteration,
        "timeseries": df_ts,
    }

    # Add all metadata columns
    for col in meta_row.columns:
        if col not in ["rank", "iter"]:
            case[col] = meta_row[col].values[0]

    cases.append(case)

print(f"Created {len(cases)} case entries")
print(f"\nExample case keys: {list(cases[0].keys())}")


# Helper function to get case by rank/iter
def get_case(rank, iteration=0):
    for case in cases:
        if case["rank"] == rank and case["iter"] == iteration:
            return case
    return None


# Display available cases
print("\nAvailable cases:")
for case in cases:
    print(f"  Rank {case['rank']}, Iter {case['iter']}")
    # Print any design variables if they exist
    for key, val in case.items():
        if key not in ["rank", "iter", "timeseries"]:
            print(f"    {key}: {val}")

## MBC Transformation Function with Low-Pass Filter

In [ ]:
from scipy.signal import butter, filtfilt


def mbc_transform(
    blade1, blade2, blade3, azimuth_deg, filter_cutoff_hz=0.5, sample_rate_hz=40
):
    """
    Multi-Blade Coordinate transformation: rotating frame → fixed frame.

    Transforms individual blade signals to collective/tilt/yaw components.

    Parameters
    ----------
    blade1, blade2, blade3 : array_like
        Signal from each blade (deflection, pitch, etc.)
    azimuth_deg : array_like
        Rotor azimuth angle in degrees (blade 1 position)
    filter_cutoff_hz : float, optional
        Low-pass filter cutoff frequency in Hz (default: 0.5 Hz)
        Set to None to disable filtering
    sample_rate_hz : float, optional
        Sampling rate of the input signals in Hz (default: 40 Hz)

    Returns
    -------
    dict with keys:
        'collective': Collective mode (symmetric, all blades together)
        'tilt': Tilt mode (fore-aft motion in fixed frame)
        'yaw': Yaw mode (side-to-side motion in fixed frame)

    Notes
    -----
    MBC transformation for 3-bladed rotor:
    - Blade phase angles: blade1=0°, blade2=120°, blade3=240°
    - Collective = (b1 + b2 + b3) / 3
    - Tilt = (2/3) * [b1*cos(ψ) + b2*cos(ψ+120°) + b3*cos(ψ+240°)]
    - Yaw = (2/3) * [b1*sin(ψ) + b2*sin(ψ+120°) + b3*sin(ψ+240°)]
    where ψ is the azimuth angle.
    """
    # Convert to numpy arrays
    b1 = np.asarray(blade1)
    b2 = np.asarray(blade2)
    b3 = np.asarray(blade3)
    psi = np.deg2rad(azimuth_deg)

    # Blade phase offsets (blade 1 at 0°, blade 2 at 120°, blade 3 at 240°)
    psi1 = psi
    psi2 = psi + np.deg2rad(120)
    psi3 = psi + np.deg2rad(240)

    # MBC transformation
    collective = (b1 + b2 + b3) / 3.0
    tilt = (2.0 / 3.0) * (b1 * np.cos(psi1) + b2 * np.cos(psi2) + b3 * np.cos(psi3))
    yaw = (2.0 / 3.0) * (b1 * np.sin(psi1) + b2 * np.sin(psi2) + b3 * np.sin(psi3))

    # Apply low-pass filter if requested
    if filter_cutoff_hz is not None:
        # Design 2nd-order Butterworth filter
        nyquist = sample_rate_hz / 2.0
        normalized_cutoff = filter_cutoff_hz / nyquist
        b_coeff, a_coeff = butter(2, normalized_cutoff, btype="low")

        # Apply zero-phase filter (forward-backward)
        collective = filtfilt(b_coeff, a_coeff, collective)
        tilt = filtfilt(b_coeff, a_coeff, tilt)
        yaw = filtfilt(b_coeff, a_coeff, yaw)

    return {
        "collective": collective,
        "tilt": tilt,
        "yaw": yaw,
    }


# Test the function with dummy data
print("MBC transformation function created ✓")
print("Usage: mbc_transform(blade1, blade2, blade3, azimuth_deg, filter_cutoff_hz=0.5)")

## Helper Function for Azimuth Wraparound

In [ ]:
def insert_nans_at_wraparound(azimuth, signal, threshold=180):
    """
    Insert NaN values where azimuth wraps from 360° to 0° to prevent line artifacts.

    Parameters
    ----------
    azimuth : array_like
        Azimuth angle array in degrees
    signal : array_like
        Signal to plot (same length as azimuth)
    threshold : float, optional
        Minimum angle jump to consider as wraparound (default: 180°)

    Returns
    -------
    azimuth_fixed, signal_fixed : numpy arrays with NaN inserted at wraparound points
    """
    az = np.asarray(azimuth)
    sig = np.asarray(signal)

    # Find large negative jumps (360→0 wraparound)
    daz = np.diff(az)
    wraparound_idx = np.where(daz < -threshold)[0]

    # Insert NaN after each wraparound point
    az_list = list(az)
    sig_list = list(sig)

    for idx in reversed(wraparound_idx):  # Reverse to maintain indices
        az_list.insert(idx + 1, np.nan)
        sig_list.insert(idx + 1, np.nan)

    return np.array(az_list), np.array(sig_list)


print("Wraparound helper function created ✓")

## Case Selection Helper

In [ ]:
def select_case(ref, zero_yaw):
    """
    Select a case based on TCIPC parameters.

    Parameters
    ----------
    ref : float
        TCIPC_MaxTipDeflection value to match
    zero_yaw : int
        TCIPC_ZeroYawDeflection value to match (0 or 1)

    Returns
    -------
    dict
        Case dictionary matching the criteria, or None if not found
    """
    for case in cases:
        if (
            case.get("TCIPC_MaxTipDeflection") == ref
            and case.get("TCIPC_ZeroYawDeflection") == zero_yaw
        ):
            return case

    raise ValueError(f"No case with {ref=}, {zero_yaw=}")

## Select Cases to Plot

Modify this dictionary to easily select which cases to compare.
Keys become the legend labels, values are selected using select_case().

In [ ]:
# Select cases to plot - keys are labels, values are selected cases
cases_to_plot = {
    "Baseline": select_case(ref=20, zero_yaw=0),
    "TCIPC": select_case(ref=10, zero_yaw=0),
    "Zero yaw": select_case(ref=10, zero_yaw=1),
}

## Plot 1: Azimuth vs Blade 1 Tip Deflection

In [ ]:
# Create plot
fig, ax = plt.subplots()

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract data
    azimuth = df["Azimuth"].values
    tipdxc1 = df["TipDxc1"].values

    # Handle wraparound
    az_fixed, tip_fixed = insert_nans_at_wraparound(azimuth, tipdxc1)

    # Plot with label
    ax.plot(az_fixed, tip_fixed, label=label)

ax.axvline(180, color="k", linestyle=":", label="Tower passing")
ax.set_xlabel("Azimuth (deg)")
ax.set_ylabel("Tip Deflection - Blade 1 (m)")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

## Plot 2: Azimuth vs Blade 1 Pitch

In [ ]:
# Create plot
fig, ax = plt.subplots()

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract data
    azimuth = df["Azimuth"].values
    bldpitch1 = df["BldPitch1"].values

    # Handle wraparound
    az_fixed, pitch_fixed = insert_nans_at_wraparound(azimuth, bldpitch1)

    # Plot with label
    ax.plot(az_fixed, pitch_fixed, label=label)

ax.axvline(180, color="k", linestyle=":", label="Tower passing")
ax.set_xlabel("Azimuth (deg)")
ax.set_ylabel("Blade Pitch - Blade 1 (deg)")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

## Plot 3: MBC Tilt vs Yaw Deflection

In [ ]:
# Create plot
fig, ax = plt.subplots()

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract tip deflection for all three blades
    azimuth = df["Azimuth"].values
    tipdxc1 = df["TipDxc1"].values
    tipdxc2 = df["TipDxc2"].values
    tipdxc3 = df["TipDxc3"].values

    # Apply MBC transformation
    mbc_deflection = mbc_transform(
        tipdxc1, tipdxc2, tipdxc3, azimuth, filter_cutoff_hz=0.3
    )

    # Plot with label
    ax.plot(mbc_deflection["yaw"], mbc_deflection["tilt"], label=label)

ax.axhline(0, color="k", linestyle="--")
ax.axvline(0, color="k", linestyle="--")
ax.set_xlabel("Yaw Deflection (m)")
ax.set_ylabel("Tilt Deflection (m)")
ax.grid()
ax.legend()
ax.set_aspect("equal", adjustable="box")
plt.tight_layout()
plt.show()

## Plot 4: MBC Tilt vs Yaw Pitch

Note, the pitch is not purely tilt even when the yaw is free because of the
azimuth offset.

In [ ]:
# Create plot
fig, ax = plt.subplots()

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract blade pitch for all three blades
    azimuth = df["Azimuth"].values
    bldpitch1 = df["BldPitch1"].values
    bldpitch2 = df["BldPitch2"].values
    bldpitch3 = df["BldPitch3"].values

    # Apply MBC transformation
    mbc_pitch = mbc_transform(
        bldpitch1, bldpitch2, bldpitch3, azimuth, filter_cutoff_hz=1
    )

    # Plot with label
    ax.plot(mbc_pitch["yaw"], mbc_pitch["tilt"], label=label)

ax.axhline(0, color="k", linestyle="--")
ax.axvline(0, color="k", linestyle="--")
ax.set_xlabel("Yaw Pitch (deg)")
ax.set_ylabel("Tilt Pitch (deg)")
ax.grid()
ax.legend()
ax.set_aspect("equal", adjustable="box")
plt.tight_layout()
plt.show()

## Plot 5: Time-Series of Tilt Deflection

In [ ]:
# Create plot
fig, ax = plt.subplots()

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract data
    time = df["Time"].values
    azimuth = df["Azimuth"].values
    tipdxc1 = df["TipDxc1"].values
    tipdxc2 = df["TipDxc2"].values
    tipdxc3 = df["TipDxc3"].values

    # Apply MBC transformation
    mbc_deflection = mbc_transform(
        tipdxc1, tipdxc2, tipdxc3, azimuth, filter_cutoff_hz=None
    )

    # Plot with label
    ax.plot(time, mbc_deflection["tilt"], label=label)

ax.set_xlabel("Time (s)")
ax.set_ylabel("Tilt Deflection (m)")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

## Torque plots

In [ ]:
# Create half-width figure
default_fig_width = plt.rcParams["figure.figsize"][0]
fig, ax = plt.subplots(figsize=(default_fig_width / 2, 2.5))

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract tip deflection for all three blades
    azimuth = df["Azimuth"].values
    tipdxc1 = df["TipDxc1"].values
    tipdxc2 = df["TipDxc2"].values
    tipdxc3 = df["TipDxc3"].values

    # Apply MBC transformation
    mbc_deflection = mbc_transform(
        tipdxc1, tipdxc2, tipdxc3, azimuth, filter_cutoff_hz=0.3
    )

    # Compute mean values
    mean_yaw = np.mean(mbc_deflection["yaw"])
    mean_tilt = np.mean(mbc_deflection["tilt"])

    # Scatter plot with mean values
    ax.scatter(mean_yaw, mean_tilt, label=label, s=50)

ax.set_xlim((-0.5, 1.0))
ax.set_ylim((0.0, 3.5))
# ax.set_xticks(np.arange(-0.5, 1.5, 0.5))
# ax.set_yticks(np.arange(-0.5, 3.5, 0.5))
ax.set_xlabel("Yaw deflection (m)")
ax.set_ylabel("Tilt deflection (m)")
ax.grid()
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
# ax.legend()
ax.set_aspect("equal", adjustable="box")
plt.tight_layout()
plt.show()

plt.savefig("../figures/tilt_yaw_deflection.pdf")

In [ ]:
# Create half-width figure
fig, ax = plt.subplots(figsize=(default_fig_width / 2, 2.5))

for label, case in cases_to_plot.items():
    df = case["timeseries"]

    # Extract data
    azimuth = df["Azimuth"].values
    tipdxc1 = df["TipDxc1"].values

    # Filter to one rotation (first occurrence of 0-360 range)
    # Find where we have a complete rotation
    start_idx = None
    end_idx = None
    for i in range(len(azimuth) - 1):
        if azimuth[i] < 10 and start_idx is None:
            start_idx = i
        if start_idx is not None and azimuth[i] > 350 and azimuth[i + 1] < 10:
            end_idx = i + 1
            break

    if start_idx is not None and end_idx is not None:
        azimuth_single = azimuth[start_idx:end_idx]
        tipdxc1_single = tipdxc1[start_idx:end_idx]
    else:
        # Fallback: just take first 360 degrees worth
        mask = azimuth <= 360
        azimuth_single = azimuth[mask]
        tipdxc1_single = tipdxc1[mask]

    # Plot single rotation
    ax.plot(azimuth_single, tipdxc1_single, label=label)

ax.axvline(180, color="k", linestyle="--", label="Tower")
ax.set_xlabel("Azimuth (deg)")
ax.set_ylabel("Tip deflection of blade 1 (m)")
ax.set_xticks(np.arange(0, 361, 60))
ax.set_ylim((9, 20))
ax.set_xlim((0, 360))
ax.grid()
ax.legend(loc="upper right", framealpha=0.9)
plt.tight_layout()
plt.show()

plt.savefig("../figures/blade1_deflection_azimuth.pdf")